In [ ]:
import asyncio
import nest_asyncio
import requests
import logging
from telegram import Update, InlineKeyboardButton, InlineKeyboardMarkup
from telegram.ext import Application, CommandHandler, CallbackQueryHandler, ContextTypes, MessageHandler, filters

# Настройка логирования
logging.basicConfig(format='%(asctime)s - %(name)s - %(levelname)s - %(message)s', level=logging.INFO)
logger = logging.getLogger(__name__)

# Токены и настройки API
TELEGRAM_TOKEN = '7117120662:AAHCqSWYlDcfpEMrIbt17_SGhRXZkImuzQs'
groq_api = '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'

# Применяем nest_asyncio для поддержки асинхронного выполнения
nest_asyncio.apply()

# Темы игры
THEMES = ["Любовь", "Дружба", "Семейный", "Уличный"]

# Словари для хранения информации об играх и текущих игроках
games = {}
current_player_index = {}

# Команда /start для начала игры и выбора тематики
async def start(update: Update, context: ContextTypes.DEFAULT_TYPE) -> None:
    chat_id = update.message.chat_id

    # Сброс старой игры, если она существует
    if chat_id in games:
        del games[chat_id]
        del current_player_index[chat_id]
        await update.message.reply_text("Старая игра сброшена. Выберите тематику игры:")
    
    # Создаем клавиатуру с доступными темами
    keyboard = [[InlineKeyboardButton(theme, callback_data=theme) for theme in THEMES]]
    reply_markup = InlineKeyboardMarkup(keyboard)
    await update.message.reply_text("Выберите тематику игры:(если вы хотите свою, то нажмите /i_want_to_have_my_game)", reply_markup=reply_markup)

# Обработка выбора темы
async def theme_selection(update: Update, context: ContextTypes.DEFAULT_TYPE) -> None:
    query = update.callback_query
    await query.answer()
    theme = query.data
    chat_id = query.message.chat_id

    # Сохраняем выбранную тему игры
    games[chat_id] = {'theme': theme, 'players': []}
    await query.edit_message_text(text=f"Вы выбрали тему: {theme}. Используйте /join_game для присоединения к игре.")

# Команда /join_game для присоединения к игре
async def join_game(update: Update, context: ContextTypes.DEFAULT_TYPE) -> None:
    chat_id = update.message.chat_id
    user_id = update.message.from_user.id

    # Проверка на наличие активной игры
    if chat_id not in games:
        await update.message.reply_text("Сначала выберите тему с помощью /start.")
        return

    game_info = games[chat_id]
    username = update.message.from_user.username

    # Добавляем игрока в список участников, если его еще нет
    if username not in game_info['players']:
        game_info['players'].append(username)
        await update.message.reply_text(f"Вы присоединились к игре! Участники: {', '.join(game_info['players'])}. Напишите /start_game для начала игры.")
    else:
        await update.message.reply_text("Вы уже зарегистрированы в игре.")

# Команда /start_game для начала игры
async def start_game(update: Update, context: ContextTypes.DEFAULT_TYPE) -> None:
    chat_id = update.message.chat_id
    # Проверка на наличие активной игры и регистрацию игрока
    if chat_id in games:
        if update.message.from_user.username in games[chat_id]['players']:
            current_player_index[chat_id] = 0 
            if len(games[chat_id]['players']) < 1:
                await update.message.reply_text("Недостаточно участников для начала игры. Зарегистрируйтесь больше.")
                return

            await update.message.reply_text(f"Игра начинается! Участники: {', '.join(games[chat_id]['players'])}.")
            await ask_question(update.message.chat_id, context)
        else:
            await update.message.reply_text("Вы не зарегистрированы в игре. Используйте /join_game, чтобы присоединиться.")
    else:
        await update.message.reply_text("Сначала выберите тему с помощью /start.")
        
# Команда /view_players для получения списка кто играет
async def view_players(update: Update, context: ContextTypes.DEFAULT_TYPE) -> None:
    chat_id = update.message.chat_id
    if chat_id in games:
        players = games[chat_id]['players']  # Получаем список игроков из словаря games
        await update.message.reply_text(f"Участники игры: {', '.join(players)}")
    else:
        await update.message.reply_text("Нет активной игры.")

        
# Обработка выбора 'Правда' или 'Действие'
async def task_selection(update: Update, context: ContextTypes.DEFAULT_TYPE) -> None:
    query = update.callback_query
    await query.answer()
    task_type, chat_id = query.data.split('_')
    chat_id = int(chat_id)
    players = games[chat_id]['players']
    current_index = current_player_index[chat_id]
    current_player = players[current_index]

    # Проверка на очередность хода
    if query.from_user.username != current_player:
        await context.bot.send_message(chat_id=query.message.chat_id, text="Это не ваш ход!")
        return
    
    # Получаем задание в зависимости от выбора игрока
    await get_task(query, context, task_type)
    current_player_index[query.message.chat_id] = (current_player_index[query.message.chat_id] + 1) % len(players)
    await ask_question(query.message.chat_id, context)

# Запрос задания для текущего игрока
async def ask_question(chat_id: int, context: ContextTypes.DEFAULT_TYPE) -> None:
    if chat_id not in games:
        await context.bot.send_message(chat_id=chat_id, text="Нет активной игры. Начните игру с помощью /start.")
        return

    players = games[chat_id]['players']
    current_index = current_player_index[chat_id]
    current_player = players[current_index]

    try:
        # Отправляем сообщение с выбором 'Правда' или 'Действие'
        await context.bot.send_message(chat_id=chat_id, text=f"@{current_player}, выберите: Правда или Действие?", reply_markup=InlineKeyboardMarkup([
            [InlineKeyboardButton("Правда", callback_data=f'truth_{chat_id}'), InlineKeyboardButton("Действие", callback_data=f'action_{chat_id}')]
        ]))
    except Exception as e:
        print(f"Ошибка при отправке сообщения игроку {current_player}: {e}")

# Получение задания от API
async def get_task(query: Update, context: ContextTypes.DEFAULT_TYPE, task_type: str) -> None:
    chat_id = query.message.chat_id

    if chat_id not in games:
        await context.bot.send_message(chat_id=chat_id, text="Сначала выберите тему с помощью /start.")
        return

    theme = games[chat_id]['theme']
    task = await generate_task(theme, task_type)
    await context.bot.send_message(chat_id=chat_id, text=task)

# Команда /end_game для завершения игры
async def end_game(update: Update, context: ContextTypes.DEFAULT_TYPE) -> None:
    chat_id = update.message.chat_id
    if chat_id in games:
        del games[chat_id]
        del current_player_index[chat_id]
        await update.message.reply_text("Игра завершена!")
    else:
        await update.message.reply_text("Нет активной игры.")

# Генерация задания через API Groq
async def generate_task(theme: str, task_type: str) -> str:
    url = "https://api.groq.com/openai/v1/chat/completions"
    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {groq_api}"
    }
    
    prompt = f"У меня есть игра, в которой человек должен ли признаться в чем-то или сделать забавное действие. Сгенерируй задание типа '{task_type}' на тему '{theme}' на русском языке, небольшое предложение."
    data = {
        "model": "mixtral-8x7b-32768",
        "messages": [{"role": "user", "content": prompt}],
        "temperature": 1,
        "max_tokens": 100
    }
    
    response = requests.post(url, headers=headers, json=data)
    if response.status_code == 200:
        return response.json()['choices'][0]['message']['content']
    else:
        return f"Ошибка при получении задания: {response.status_code}"

# Команда для установки пользовательской темы
async def set_custom_theme(update: Update, context: ContextTypes.DEFAULT_TYPE) -> None:
    await update.message.reply_text("Напишите тему, по которой вы хотите играть:")
    context.user_data['waiting_for_theme'] = True

# Обработка пользовательской темы
async def handle_custom_theme(update: Update, context: ContextTypes.DEFAULT_TYPE) -> None:
    if context.user_data.get('waiting_for_theme'):
        theme = update.message.text
        chat_id = update.message.chat_id
        games[chat_id] = {'theme': theme, 'players': []}
        context.user_data['waiting_for_theme'] = False
        
        await update.message.reply_text(f"Вы выбрали тему: {theme}. Используйте /join_game для присоединения к игре.")

# Команда /help для получения справки
async def help_command(update: Update, context: ContextTypes.DEFAULT_TYPE) -> None:
    help_text = (
        "Привет! Я бот для игры 'Правда или действие'. Вот как ты можешь использовать меня:\n\n"
        "/start - Начать игру и выбрать тематику.\n"
        "/i_want_to_have_my_game - Начать игру и написать свою тематику\n"
        "/join_game - Присоединиться к игре после выбора тематики.\n"
        "/start_game - Начать игру с участниками.\n"
        "/view_players - Посмотрите список участников текущей игры.\n"
        "/end_game - Завершить текущую игру.\n"
        "/help - Показать это сообщение.\n\n"
        "После начала игры каждый игрок будет по очереди выбирать между 'Правда' и 'Действие'. "
        "Удачи и веселитесь!"
    )
    await update.message.reply_text(help_text)

# Основная функция для запуска бота
def main():
    application = Application.builder().token(TELEGRAM_TOKEN).build()

    # Регистрация обработчиков команд
    application.add_handler(CommandHandler("start", start))
    application.add_handler(CommandHandler("join_game", join_game))
    application.add_handler(CommandHandler("start_game", start_game))
    application.add_handler(CommandHandler("end_game", end_game))
    application.add_handler(CommandHandler("i_want_to_have_my_game", set_custom_theme))
    application.add_handler(CommandHandler("help", help_command))
    application.add_handler(CommandHandler("view_players", view_players))
    
    # Регистрация обработчиков нажатий кнопок
    application.add_handler(CallbackQueryHandler(theme_selection, pattern='^(Любовь|Дружба|Семейный|Уличный)$'))
    application.add_handler(CallbackQueryHandler(task_selection, pattern='^(truth|action)_'))
    
    # Обработка текстовых сообщений для установки пользовательской темы
    application.add_handler(MessageHandler(filters.TEXT & ~filters.COMMAND, handle_custom_theme))

    # Запуск бота
    application.run_polling()

# Запуск основной функции
if __name__ == "__main__":
    main()

2024-10-24 21:27:22,303 - httpx - INFO - HTTP Request: POST https://api.telegram.org/bot7117120662:AAHCqSWYlDcfpEMrIbt17_SGhRXZkImuzQs/getMe "HTTP/1.1 200 OK"
2024-10-24 21:27:22,385 - httpx - INFO - HTTP Request: POST https://api.telegram.org/bot7117120662:AAHCqSWYlDcfpEMrIbt17_SGhRXZkImuzQs/deleteWebhook "HTTP/1.1 200 OK"
2024-10-24 21:27:22,388 - telegram.ext.Application - INFO - Application started
2024-10-24 21:27:22,654 - httpx - INFO - HTTP Request: POST https://api.telegram.org/bot7117120662:AAHCqSWYlDcfpEMrIbt17_SGhRXZkImuzQs/getUpdates "HTTP/1.1 200 OK"
2024-10-24 21:27:22,821 - httpx - INFO - HTTP Request: POST https://api.telegram.org/bot7117120662:AAHCqSWYlDcfpEMrIbt17_SGhRXZkImuzQs/sendMessage "HTTP/1.1 200 OK"
2024-10-24 21:27:25,170 - httpx - INFO - HTTP Request: POST https://api.telegram.org/bot7117120662:AAHCqSWYlDcfpEMrIbt17_SGhRXZkImuzQs/getUpdates "HTTP/1.1 200 OK"
2024-10-24 21:27:25,322 - httpx - INFO - HTTP Request: POST https://api.telegram.org/bot7117120662:AA